In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

In [2]:
url = 'https://github.com/matheusfacure/causal-inference-in-python-code/blob/main/causal-inference-in-python/data/rec_ab_test.csv?raw=true'

df = pd.read_csv(url)
df.tail()

,recommender,age,tenure,watch_time
318,benchmark,18,0,2.13
319,benchmark,16,0,1.70
320,benchmark,15,0,1.25
321,benchmark,16,2,3.31
322,benchmark,24,1,1.82


In [3]:
result = smf.ols("watch_time ~C(recommender)", data=df).fit()

In [5]:
result.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,2.0491,0.058,35.367,0.000,1.935,2.163
C(recommender)[T.challenger],0.1427,0.095,1.501,0.134,-0.044,0.330


- Intercept: estimate for $\beta_0$. We can interpret it as the expected watch time for those who receive the control (`recommender = 0`)
- $\hat{\beta}_1$: the increase/decrease in watch time due to the alternative (`recommender = 1`). In other words, it is the estimate for the ATE.

In [6]:
df.groupby("recommender")["watch_time"].mean()

recommender
benchmark     2.049064
challenger    2.191750
Name: watch_time, dtype: float64

In [8]:
url = 'https://github.com/matheusfacure/causal-inference-in-python-code/blob/main/causal-inference-in-python/data/risk_data.csv?raw=true'

df_risk = pd.read_csv(url)
df_risk.tail()

,wage,educ,exper,married,credit_score1,credit_score2,credit_limit,default
49995,840.0,12,13,1,466.0,482.0,2400.0,0
49996,700.0,15,16,1,328.0,393.0,1100.0,0
49997,930.0,14,16,1,552.0,339.0,1700.0,0
49998,1550.0,17,7,1,569.0,536.0,3300.0,0
49999,1170.0,12,18,1,466.0,518.0,3200.0,0


`credit_limit` has too many categories. In this case, it is better to treat is as a continuous variable and representing the ATE as the derivative of the expected outcome with respect to the treatment:

$$ATE = \frac{\partial}{\partial t} \mathbb{E}[y | t]$$

In other words, it is the amount we expect the outcome to change given a unit increase in the treatment

In [9]:
model = smf.ols("default ~credit_limit", data=df_risk).fit()
model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.2192,0.004,59.715,0.000,0.212,0.226
credit_limit,-2.402e-05,1.16e-06,-20.689,0.000,-2.63e-05,-2.17e-05


Instead of manually adjusting for the confounders, we can simply add them to the model:

$$Default_i = \beta_0 + \beta_1 \cdot limit_i + \theta_1 \cdot wage_i + \theta_2 \cdot creditScore1_i + \theta_3 \cdot creditScore2_i + e_i$$

In [10]:
formula = "default ~ credit_limit + wage + credit_score1 + credit_score2"
model = smf.ols(formula, data=df_risk).fit()
model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.4037,0.009,46.939,0.000,0.387,0.421
credit_limit,3.063e-06,1.54e-06,1.987,0.047,4.16e-08,6.08e-06
wage,-8.822e-05,6.07e-06,-14.541,0.000,-0.000,-7.63e-05
credit_score1,-4.175e-05,1.83e-05,-2.278,0.023,-7.77e-05,-5.82e-06
credit_score2,-0.0003,1.52e-05,-20.055,0.000,-0.000,-0.000


In [42]:
debiasing_model = smf.ols("credit_limit ~ wage + credit_score1 + credit_score2", data=df_risk).fit()

df_risk_deb = df_risk.assign(
    # for visualization, avg(T) is added to the residuals
    credit_limit_res = debiasing_model.resid + df_risk["credit_limit"].mean()
)

df_risk_deb

,wage,educ,exper,married,credit_score1,credit_score2,credit_limit,default,credit_limit_res
0,950.0,11,16,1,500.0,518.0,3200.0,0,3347.583906
1,780.0,11,7,1,414.0,429.0,1700.0,0,2331.086567
2,1230.0,14,9,1,586.0,571.0,4200.0,0,3665.134174
3,1040.0,15,8,1,379.0,411.0,1500.0,0,1720.933260
4,1000.0,16,1,1,379.0,518.0,1800.0,0,2084.659827
...,...,...,...,...,...,...,...,...,...
49995,840.0,12,13,1,466.0,482.0,2400.0,0,2817.487623
49996,700.0,15,16,1,328.0,393.0,1100.0,0,2044.247688
49997,930.0,14,16,1,552.0,339.0,1700.0,0,1802.366493
49998,1550.0,17,7,1,569.0,536.0,3300.0,0,2212.155035


In [43]:
model_w_deb_data = smf.ols("default ~ credit_limit_res", data=df_risk_deb).fit()
model_w_deb_data.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.1421,0.005,30.001,0.000,0.133,0.151
credit_limit_res,3.063e-06,1.56e-06,1.957,0.050,-4.29e-09,6.13e-06


In [44]:
denoising_model = smf.ols("default ~ wage + credit_score1 + credit_score2", data=df_risk_deb).fit()
df_risk_denoise = df_risk_deb.assign(default_res = denoising_model.resid + df_risk_deb["default"].mean())

df_risk_denoise

,wage,educ,exper,married,credit_score1,credit_score2,credit_limit,default,credit_limit_res,default_res
0,950.0,11,16,1,500.0,518.0,3200.0,0,3347.583906,0.000981
1,780.0,11,7,1,414.0,429.0,1700.0,0,2331.086567,-0.043175
2,1230.0,14,9,1,586.0,571.0,4200.0,0,3665.134174,0.043289
3,1040.0,15,8,1,379.0,411.0,1500.0,0,1720.933260,-0.028427
4,1000.0,16,1,1,379.0,518.0,1800.0,0,2084.659827,0.000760
...,...,...,...,...,...,...,...,...,...,...
49995,840.0,12,13,1,466.0,482.0,2400.0,0,2817.487623,-0.020257
49996,700.0,15,16,1,328.0,393.0,1100.0,0,2044.247688,-0.063805
49997,930.0,14,16,1,552.0,339.0,1700.0,0,1802.366493,-0.053155
49998,1550.0,17,7,1,569.0,536.0,3300.0,0,2212.155035,0.058478


In [46]:
model_se = smf.ols("default ~ wage + credit_score1 + credit_score2", data=df_risk).fit()

print ("SE regression:", model_se.bse["wage"])

model_wage_aux = smf.ols("wage ~ credit_score1 + credit_score2", data=df_risk).fit()

# subtract the degrees of freedom - 4 model parameterrs - from N
se_formula = (np.std(model_se.resid) / np.std(model_wage_aux.resid) * np.sqrt(len(df_risk) - 4))
print ("SE formula:", se_formula)

SE regression: 5.364242347548205e-06
SE formula: 0.26819066040801987


In [47]:
model_w_orthogonal = smf.ols("default_res ~ credit_limit_res", data=df_risk_denoise).fit()
model_w_orthogonal.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.1421,0.005,30.458,0.000,0.133,0.151
credit_limit_res,3.063e-06,1.54e-06,1.987,0.047,4.17e-08,6.08e-06
